In [55]:
import argparse
import time
import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler


In [56]:
import os
import torch

from collections import Counter


class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []
        self.counter = Counter()
        self.total = 0

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        token_id = self.word2idx[word]
        self.counter[token_id] += 1
        self.total += 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))

    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r') as f:
            ids = torch.LongTensor(tokens)
            token = 0
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1

        return ids


In [57]:
def batchify(data, bsz, args):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    #if args.cuda:
        #data = data.cuda()
    return data

In [58]:
def evaluate(data_source, batch_size=10):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    if args.model == 'QRNN': model.reset()
    total_loss = 0
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(batch_size)
    for i in range(0, data_source.size(0) - 1, args.bptt):
        #print('eval:',i)
        #print('before get_batch,  max_memory_reserved():',torch.cuda.max_memory_reserved()/1024/1024)
        data, targets = get_batch(data_source, i, args, evaluation=True)
        #print('after get_batch,  max_memory_reserved():',torch.cuda.max_memory_reserved()/1024/1024)
        output, hidden = model(data, hidden)
        total_loss += len(data) * criterion(model.decoder.weight, model.decoder.bias, output, targets).data
        hidden = repackage_hidden(hidden)
    return total_loss.item() / len(data_source)

In [59]:
#print('Loading cached dataset...')
#corpus = torch.load('corpus.148650ff682fa3f76e78c18d7d6d5bd6.data')

print('Producing dataset...')
corpus = Corpus('data/penn/')
#torch.save(corpus, fn)

Producing dataset...


In [60]:
test_batch_size = 1
train_batch_size = 20
#args = {'cuda':True}
class arg1:
    cuda = True
    model  = 'LSTM'
    bptt = 70
args = arg1()
test_data = batchify(corpus.test, test_batch_size, args)
train_data = batchify(corpus.train, train_batch_size, args)

In [61]:
print('The size of test data = ', test_data.shape)
print('The size of train data = ',train_data.shape)

The size of test data =  torch.Size([82430, 1])
The size of train data =  torch.Size([46479, 20])


In [48]:
def model_load(fn):
    global model, criterion, optimizer
    #if args.philly:
        #fn = os.path.join(os.environ['PT_OUTPUT_DIR'], fn)
    #fn = '1591009087197056.pt'
    with open(fn, 'rb') as f:
        model, criterion, optimizer = torch.load(f, map_location=torch.device('cpu'))

In [50]:
model_load('model/model-{}'.format(105))

In [51]:
test_loss = evaluate(test_data, test_batch_size)

In [52]:
val_loss

4.638208555744268